In [0]:
# predict what passengers survived the titanic crash based on age, cabin, children, etc.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logreg_example').getOrCreate()
df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/sejal@ibm.com/titanic.csv", inferSchema=True, header=True)

In [0]:
my_cols = df.select(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])
my_cols.show()

+--------+------+------+----+-----+-----+-------+--------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
 0| 3| male|22.0| 1| 0| 7.25| S|
 1| 1|female|38.0| 1| 0|71.2833| C|
 1| 3|female|26.0| 0| 0| 7.925| S|
 1| 1|female|35.0| 1| 0| 53.1| S|
 0| 3| male|35.0| 0| 0| 8.05| S|
 0| 3| male|null| 0| 0| 8.4583| Q|
 0| 1| male|54.0| 0| 0|51.8625| S|
 0| 3| male| 2.0| 3| 1| 21.075| S|
 1| 3|female|27.0| 0| 2|11.1333| S|
 1| 2|female|14.0| 1| 0|30.0708| C|
 1| 3|female| 4.0| 1| 1| 16.7| S|
 1| 1|female|58.0| 0| 0| 26.55| S|
 0| 3| male|20.0| 0| 0| 8.05| S|
 0| 3| male|39.0| 1| 5| 31.275| S|
 0| 3|female|14.0| 0| 0| 7.8542| S|
 1| 2|female|55.0| 0| 0| 16.0| S|
 0| 3| male| 2.0| 4| 1| 29.125| Q|
 1| 2| male|null| 0| 0| 13.0| S|
 0| 3|female|31.0| 1| 0| 18.0| S|
 1| 3|female|null| 0| 0| 7.225| C|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 20 rows

In [0]:
my_final_data = my_cols.na.drop()

In [0]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [0]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [0]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [0]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'],
                           outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [0]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [0]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [0]:
fit_model = pipeline.fit(train_data)

In [0]:
results = fit_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [0]:
results.select('Survived', 'prediction').show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [0]:
AUC = my_eval.evaluate(results)
AUC

Out[18]: 0.7905374887082204